# Impulse Response

## Table of Contents

- [Background](#background)
- [Finite Impulse Response](#finite-impulse-response)
- [Infinite Impulse Response](#infinite-impulse-response)


## Background

In [20]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig

from ipywidgets import (
    FloatSlider, IntSlider, Dropdown, Checkbox,
    HBox, VBox, Layout
)
from ipywidgets import interactive_output

plt.rcParams.update({
    "font.family": "serif",
    "mathtext.fontset": "cm",
    "axes.unicode_minus": False
})

## Finite Impulse Response

In [24]:
# =============================================================================
# Interactive FIR Filter (Matplotlib + ipywidgets)
# -----------------------------------------------------------------------------
# Layout + style goals:
#   - Sliders across the top (two rows)
#   - Big centered title
#   - Light-blue rounded formula box inside axes (top-left)
#   - Parameter readout inside axes under the formula
#   - Clear plot with grid and nice line weight
#
# What this shows:
#   - Time-domain input x[n] and FIR output y[n] (main plot)
#   - Vertical dashed line indicating "cursor" sample n0
#   - Optional: show the FIR impulse response h[k] as a small inset
#
# Requirements:
#   pip install numpy scipy matplotlib ipywidgets
# =============================================================================

# =============================================================================
# Helpers
# =============================================================================
def _make_signal(kind: str, fs: float, n: int, f1: float, f2: float, noise_std: float, seed: int):
    rng = np.random.default_rng(int(seed))
    t = np.arange(n) / fs

    if kind == "sine":
        x = np.sin(2*np.pi*f1*t)
    elif kind == "sum_sines":
        x = 0.7*np.sin(2*np.pi*f1*t) + 0.3*np.sin(2*np.pi*f2*t)
    elif kind == "chirp":
        dur = t[-1] if t[-1] > 0 else 1.0
        x = sig.chirp(t, f0=max(1.0, f1), f1=max(1.0, f2), t1=dur, method="linear")
    elif kind == "step":
        x = np.zeros(n); x[n//4:] = 1.0
    elif kind == "noise":
        x = rng.normal(0.0, 1.0, size=n)
    else:
        raise ValueError("Unknown signal kind")

    if noise_std > 0:
        x = x + rng.normal(0.0, noise_std, size=n)

    return t, x


def _design_fir(mode: str, M: int, fs: float, fc1: float, fc2: float, window: str):
    nyq = fs / 2.0
    eps = nyq * 1e-6

    fc1 = float(np.clip(fc1, eps, nyq - eps))
    fc2 = float(np.clip(fc2, eps, nyq - eps))

    if mode in ("bandpass", "bandstop"):
        lo, hi = sorted([fc1, fc2])
        if hi - lo < nyq * 1e-3:
            hi = min(nyq - eps, lo + nyq * 1e-3)
        cutoff = [lo, hi]
        pass_zero = (mode == "bandstop")
    else:
        cutoff = fc1
        pass_zero = (mode == "lowpass")

    h = sig.firwin(numtaps=int(M), cutoff=cutoff, pass_zero=pass_zero, fs=fs, window=window)
    return h


# =============================================================================
# Plot function (called by interactive_output)
# =============================================================================
def plot_fir(
    fs, M, win, mode, fc1, fc2,
    x_kind, f1, f2, n, noise_std, seed,
    t0, dt, n0, show_h
):
    fs = float(fs)
    M = int(M)
    n = int(n)
    seed = int(seed)
    n0 = int(np.clip(int(n0), 0, n - 1))

    # Generate signal and filter output
    t, x = _make_signal(x_kind, fs, n, float(f1), float(f2), float(noise_std), seed)
    h = _design_fir(mode, M, fs, float(fc1), float(fc2), win)
    y = sig.lfilter(h, [1.0], x)

    # Windowed view (prevents “mess”)
    t0 = float(np.clip(t0, 0.0, t[-1]))
    dt = float(np.clip(dt, 0.01, max(0.02, t[-1] - t0)))
    t1 = t0 + dt

    i0 = int(np.clip(np.searchsorted(t, t0), 0, n - 2))
    i1 = int(np.clip(np.searchsorted(t, t1), i0 + 2, n))

    # ---- Figure ----
    fig, ax = plt.subplots(figsize=(12.5, 6.2))
    ax.set_title("FIR Filter (Finite Impulse Response)", fontsize=30, pad=14)

    ax.plot(t[i0:i1], x[i0:i1], linewidth=3, alpha=0.85, label="x(t)")
    ax.plot(t[i0:i1], y[i0:i1], linewidth=3, alpha=0.95, label="y(t)")

    ax.axvline(t[n0], linestyle="--", linewidth=2)

    ax.set_xlabel(r"$t\ (s)$", fontsize=16)
    ax.set_ylabel("amplitude", fontsize=20)
    ax.grid(True, alpha=0.25)
    ax.legend(loc="upper right", frameon=True)

    # Parameter readout (no formula)
    if mode in ("bandpass", "bandstop"):
        lo, hi = sorted([float(fc1), float(fc2)])
        cutoff_str = f"{lo:.0f}–{hi:.0f} Hz"
    else:
        cutoff_str = f"{float(fc1):.0f} Hz"

    fig.text(
        0.02, 0.06,
        rf"$M={M},\ \mathrm{{mode}}={mode},\ \mathrm{{win}}={win},\ f_s={fs:.0f}\ \mathrm{{Hz}},\ \mathrm{{cutoff}}={cutoff_str},\ y[n_0]={y[n0]:.4f}$",
        fontsize=12
    )

    # Optional inset: h[k]
    if bool(show_h):
        iax = ax.inset_axes([0.68, 0.10, 0.28, 0.28])
        k = np.arange(len(h))
        markerline, stemlines, baseline = iax.stem(k, h)
        plt.setp(stemlines, linewidth=1.5)
        plt.setp(markerline, markersize=4)
        plt.setp(baseline, linewidth=1.0)
        iax.set_title(r"$h[k]$", fontsize=12)
        iax.grid(True, alpha=0.2)
        iax.set_xlabel("k", fontsize=10)
        iax.set_ylabel("amp", fontsize=10)

    plt.show()


# =============================================================================
# Widget layout (same style logic as your Gaussian code)
# =============================================================================
_w = Layout(width="230px")
_row = Layout(justify_content="flex-start", gap="12px")
_pad = Layout(padding="0px 0px 0px 0px")

w_fs    = FloatSlider(value=2000.0, min=200.0, max=5000.0, step=1.0,   description="fs",   continuous_update=True, layout=_w)
w_M     = IntSlider(  value=101,   min=9,     max=401,   step=2,       description="M",    continuous_update=True, layout=_w)
w_win   = Dropdown(   options=["hann","hamming","blackman","boxcar"], value="hann", description="win", layout=_w)

w_mode  = Dropdown(   options=["lowpass","highpass","bandpass","bandstop"], value="lowpass", description="mode", layout=_w)
w_fc1   = FloatSlider(value=200.0, min=1.0,   max=2500.0, step=1.0,   description="fc1",  continuous_update=True, layout=_w)
w_fc2   = FloatSlider(value=500.0, min=1.0,   max=2500.0, step=1.0,   description="fc2",  continuous_update=True, layout=_w)

w_xkind = Dropdown(   options=["sum_sines","sine","chirp","step","noise"], value="sum_sines", description="x[n]", layout=_w)
w_f1    = FloatSlider(value=50.0,  min=1.0,   max=2000.0, step=1.0,   description="f1",   continuous_update=True, layout=_w)
w_f2    = FloatSlider(value=400.0, min=1.0,   max=2000.0, step=1.0,   description="f2",   continuous_update=True, layout=_w)

w_n     = IntSlider(  value=4000,  min=512,   max=20000, step=256,     description="n",    continuous_update=True, layout=_w)
w_sn    = FloatSlider(value=0.00,  min=0.00,  max=1.00,  step=0.01,    description="σn",  continuous_update=True, layout=_w)
w_seed  = IntSlider(  value=0,     min=0,     max=999,   step=1,       description="seed", continuous_update=True, layout=_w)

w_t0    = FloatSlider(value=0.00,  min=0.00,  max=2.00,  step=0.001,   description="t0",   continuous_update=True, layout=_w)
w_dt    = FloatSlider(value=0.08,  min=0.01,  max=0.50,  step=0.005,   description="Δt",   continuous_update=True, layout=_w)
w_n0    = IntSlider(  value=200,   min=0,     max=3999,  step=1,       description="n0",   continuous_update=True, layout=_w)

w_showh = Checkbox(value=True, description="h[k] inset", indent=False, layout=Layout(width="120px"))

controls = VBox(
    [
        HBox([w_fs, w_M,  w_win],  layout=_row),
        HBox([w_mode, w_fc1, w_fc2], layout=_row),
        HBox([w_xkind, w_f1, w_f2], layout=_row),
        HBox([w_n, w_sn, w_seed], layout=_row),
        HBox([w_t0, w_dt, w_n0], layout=_row),
        HBox([w_showh], layout=Layout(justify_content="flex-start")),
    ],
    layout=_pad
)

out = interactive_output(
    plot_fir,
    {
        "fs": w_fs, "M": w_M, "win": w_win,
        "mode": w_mode, "fc1": w_fc1, "fc2": w_fc2,
        "x_kind": w_xkind, "f1": w_f1, "f2": w_f2,
        "n": w_n, "noise_std": w_sn, "seed": w_seed,
        "t0": w_t0, "dt": w_dt, "n0": w_n0,
        "show_h": w_showh
    }
)

VBox([controls, out])


## Infinite Impulse Response

----
----
----

In [23]:
import sys
from pathlib import Path
import importlib

# -----------------------------------------------------------------------------
# Robustly locate: <repo>/notebooks/For_Author
# -----------------------------------------------------------------------------
def _find_for_author_dir() -> Path:
    for base in [Path.cwd(), *Path.cwd().parents]:
        candidate = base / "notebooks" / "For_Author"
        if candidate.is_dir():
            return candidate
    raise FileNotFoundError("Could not find notebooks/For_Author by walking up from cwd.")

author_tools = _find_for_author_dir()

# Put it FIRST on sys.path (important if there are name collisions)
sys.path.insert(0, str(author_tools))

# Fresh import (handles notebook re-runs cleanly)
mod = importlib.import_module("retrieve_headings")
importlib.reload(mod)

make_toc = mod.make_toc

make_toc("Impulse_Response.ipynb")

TOC copied to clipboard:

## Table of Contents

- [Impulse Response](#impulse-response)
  - [Table of Contents](#table-of-contents)
  - [Background](#background)
  - [Finite Impulse Response](#finite-impulse-response)
  - [Infinite Impulse Response](#infinite-impulse-response)
